In [2]:
import pandas as pd
import nltk
import re
data = pd.read_csv('Yelp Data Restaurant Reviews Ratings.csv')
data['category'] = 'high'
data.ix[data.stars < 4, 'category'] = 'low'

In [3]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans

#### Building a document-term matrix for positive reviews

In [4]:
pos_reviews  = data.ix[data.category == 'high']['Review']
vectorizer = CountVectorizer(min_df = 0, stop_words = 'english')
dtm = vectorizer.fit_transform(pos_reviews)

#### Perform Latent Semantic Analysis on the matrix with 5 components

In [5]:
lsa = TruncatedSVD(5, algorithm = 'randomized')
dtm_lsa = lsa.fit_transform(dtm)
dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)
pos_lsa = pd.DataFrame(dtm_lsa, index = pos_reviews, columns = ["component_1","component_2","component_3","component_4","component_5" ])

#### The reviews are ranked based on the score in individual components, then topics are extracted from the top reviews

component 1: people tend to talk about the service

In [6]:
pos_lsa.sort_values('component_1', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
absolutely delicious indian fare. can even get completely vegan. it s small inside but they turn tables quick so you shouldn t have to wait long. for our own food we waited only a few minutes before we had piping hot tasty curry on our table. we had the chickpea ceviche (recommended by our server) for an app and it was so fresh and cool and crisp tasting. then onto the bombay plate 5 picks of entrees with lettuce wraps and each thing we had was sooooo good....spinach and tofu vegetable curry eggplant curry vegetable makki and lentils. came with a little roti bread and we tore into it. for desert we could only the sorbet since the other options had allergy inducing pistachios but it was a nice cool rich tasting treat. our server was severely genuine and sincere and made us feel like valued customers. of course she was from alabama so we would ve expected nothing less. a pleasure to have her taking care of us. a great place and i know we ll be back here. 30 bucks for two with 2 large apps a large entree and desert.,0.997952,-0.017851,-0.042879,-0.036130,-0.025081
Oh man am I the only one who loves Macayo s?? They have great chips that are always served extremely quickly and delicious salsa that they will readily fill back up. Whenever my coworker and I feel like eating fatty unhealthy food we always go there for the Baja Burro - oh my gosh...so good! It is a burrito filled with either chicken steak or shrimp covered in baja sauce and cheese. Its freaking delicious! You can also get a bean and cheese burrito Baja Style if you choose and they will add all of this deliciousness to your bean and cheeser.I want to go for happy hour and try their new watermelon and honeydew margs because those sound delish. Anyway I totally recommend this place for a quick lunch I ve never been disappointed and never left even remotely hungry. In fact most times I feel I need a wheelchair.,0.996633,-0.070892,-0.027047,0.024454,0.019169
Wow. I am not a fan of casual bar type restaurants yet this place completely won me over. From the impeccable service to the fabulous food all of us had a wonderful time. I HIGHLY recommend the Mixed Grain Salad. I was worried it sounded too healthy for my tastes but luckily we had the world s best waitress and she recommended I get the Grilled Chicken Sandwich and to have the Mixed Grain Salad as the side. MMMMMM! The sandwich was really good but the salad was OMG out of this world! Next time I will just get the salad. Everyone loved their meals and the atmosphere was so relaxed. Fabulous time and will definitely be back!,0.996228,0.041795,0.045627,-0.051884,-0.031751
The Windsor drips with attitude and atmosphere. Imagine a really cool gastro pub with high quality (and deliciously savory) food items surrounded in plaid patterns retro-chic (male heavy) influenced art design and a kick-ass patio area. The bar inside offers just about anything your alcoholic heart might desire- with a killer happy hour of $5 beer pitchers (great selection of brews to boot) glasses of wine from the tap (yes I said tapped wine but it s really good stuff) and other delicacies. The staff is extremely friendly- they know when to check in and when to just let your group hang out and enjoy the Windsor. If you re not hung up on meat please try the burger Windsor style- it doesn t get any better for the $11 price- and includes one of their indulgent sides (mac n cheese for the win). Expect a crowd because the establishment is top notch- the owners know a thing or two about creating an experience as they also manage the Postino chain (one of the 3 locations is basically across the street from Windsor). If you re in the mood for a really incredible place to hang out have a few drinks indulge in some delicious food and really just enjoy a laid back dining experience then the Windsor should be at the top of your must try list. Afterwards walk next door to Churn and order up some

component 2: people like to talk about the food quality

In [7]:
pos_lsa.sort_values('component_2', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
Zupa di pesch was phenom!,0.408136,0.707116,-0.447443,0.109859,-0.348049
YUM if you like a pizza with no sauce get the Snow White! Lots of garlic cheese seasoning. Declicious!!,0.636450,0.699097,-0.225796,0.068475,0.224772
Hang on...gotta wipe the cheese off my chin. Oh crap the enchilada sauce is on my keyboard.Cheese crisp with Guacamole. Two cheese enchiladas one for tomorrow.There s a mess. Everywhere. Ahhhhhhhhhhhhhyessssssssssss!,0.608590,0.682215,-0.231227,0.037927,-0.330601
Skip the full-sized burger and order sliders; my fiance was eyeing mine jealously. Added crispy-not-oily onion rings finished with a chunky banana milkshake and left smiling full but without any greasy spoon remorse.,0.677339,0.679321,-0.258440,-0.047834,0.103228
Butterfish and chocolate souffle are pure heaven!,0.454307,0.676957,-0.287216,-0.064411,-0.498690
I ve been here dozens and DOZENS of times over the last 25 years. 5 stars without a doubt.Standouts are the Italian beef with lots o hot . Their cheese fries are absolute perfection... here s why. Put fries in a basket smother them with cheese sauce put another hand full of fries in basket SMOTHER AGAIN WITH CHEESE SAUCE.When you are done with these cheese fries you are looking for more fries for the cheese not more cheese for the fries.Then when you walk out school girls and boys break into song and birds start chirping.Oh I have to mention the cheese steak. I don t like the standard way they do it (beef onions cheese) BUT if you ask for it Philly Style they do it with their cheese sauce that they add extra cheddar cheese to.Yum yum yum.,0.697109,0.674597,-0.229345,0.000420,-0.079739
what is there to be said that hasn t already been?try their sea salt choc chip cookies. with coffee or milk. get all the FREE toppings on your pizza that you like! (egg? garlic? chilies!??!?!) my fave? the roasted corn with goat cheese. put an egg on it!,0.669554,0.673247,-0.251434,0.051697,0.180398
Wow..got a $5.00 for just for checking in..,0.678272,0.664798,-0.288232,0.033950,0.117303
Pizza is spicy and sweet. Lovely,0.620436,0.646006,-0.269260,-0.309383,-0.171806


component 3: people give have a good overall impression

In [8]:
pos_lsa.sort_values('component_3', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
So good,0.337892,0.112886,0.894306,-0.230816,-0.141515
Very good,0.337892,0.112886,0.894306,-0.230816,-0.141515
ridiculously good ...,0.340569,0.114187,0.893978,-0.227685,-0.141200
Good and unique.,0.351807,0.111375,0.889138,-0.230067,-0.142582
Good tofu!,0.349382,0.117492,0.888643,-0.227218,-0.151043
Good gyro meat. Tzatziki is good.,0.360052,0.127611,0.885955,-0.223133,-0.139189
Never been to the sky harbor location . The ones in CA are real good tho!,0.396104,0.105412,0.880792,-0.195340,-0.134305
I had Filet Mignon as usual and it was not as good as Ruth s Chris or even Sweetwater Traven in VA.,0.360757,0.132095,0.879919,-0.234477,-0.152214
I recommend the Pollo Siciliano and the Pollo basil.Good Stuff.,0.385788,0.109719,0.879017,-0.222317,-0.130510


component 4: a lot of mexican food

In [9]:
pos_lsa.sort_values('component_4', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
Chiptole.... enough saidgreat mexican for being americanized chain....massive burrito for $8,0.344951,-0.363200,0.146176,0.786056,0.331425
This is how Mexican food should be served:It should cost about five bucks. You start with some form of tortilla add refried beans spanish rice salsa and lettuce insert cheesey-messy-possibly-meaty-goodness and throw it all in a styrofoam container. Then you ravage the contents of the container until there are fork scratch marks in the styrofoam. Thank you Carolina s for being one of the few the brave and the proud Mexican places to carry out this process with charm and elegance.,0.481009,-0.391547,0.092097,0.774333,0.085135
The fluffiest rice I have ever eaten!,0.610658,0.175078,-0.010052,0.770071,-0.057746
I Heart Los!!!This is my absolute favorite Mexican Joint!! Lite A with chicken and a pepsi.....mmmmmm heaven.,0.627344,-0.024177,-0.114682,0.768858,0.039500
TOR.TI.LLAS......Delicioso :)!! Andale mi amgos.....,0.551472,-0.281917,0.036576,0.760570,0.191303
spicy new mexican! yum!,0.601088,-0.168118,-0.053792,0.760483,0.170884
One of our favorite Mexican food restaurants in the valley. Stay away from the Americanized dishes; instead go for the corn and cabbage enchiladas with while sauce egg noodles covered in green chili and assorted tamales.,0.574684,-0.324287,0.014455,0.748453,-0.064698
Hushpuppies and gumbo!,0.463683,-0.043555,0.154349,0.746996,-0.448637
Ben B. and I visited Al-Hana after a tiresome IKEA outing and BABA-GODDAMN this stuff is YUMMY AS HELL. We nearly stuffed ourselves to capacity on the smoky babaganoush and hummus. We had the chicken shawarma and Ben had something with spiced beef in it... the name lasted on my tongue about as long as the food can t remember it so sorry.Nothing over 3.99. Seriously. You must.,0.560865,-0.349949,0.045050,0.742706,-0.096565


component 5: The overall place, such as atmosphere

In [10]:
pos_lsa.sort_values('component_5', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
Everyone I have told about this place is like wtf is a torta. I have to bring them because it can not be described it has to be experienced.,0.466813,-0.048763,-0.126381,-0.224845,0.844500
I would like to know what happen to Coffee Dans restruant I heard it burnt down when did this take place. I worked there as a teenager in 1963. My email is countryman545@yahoo.com.,0.528361,0.042205,-0.163407,-0.206418,0.806066
May look like a dump I a bad neighborhood but this place is always packed at lunch. Hands down my favorite Mexican joint north of the border,0.567737,-0.103086,-0.126520,-0.085727,0.802304
The absolute best!! I love this place and have been coming here for something like 15 years even when I lived in other states (like MA and WA and CA).,0.558833,0.043102,-0.193330,-0.071406,0.802105
Oye que tal mis amigos? This is the premier place if you want to find out what beef chimichangas should taste like. The place has all the charm of an abandoned warehouse but the eats are second-to-none. Take some tortillas home you ll be glad you did!Been eating at this place faithfully for the past five years and the quality has always been consistent. I get it to go but sat in a few times to eat as well. South Phoenix and close to Van Buren what s not to like?,0.537565,-0.085740,-0.149477,-0.206730,0.799119
Bomba! Wow.. I m here eating while I draft this review. Don t sleep on this place!!!,0.432508,-0.229202,-0.109362,-0.354035,0.789368
Trust the positive reviews... This place is like no other. When I travel for work to Phoenix this is the first place I think about whether I get there or not. I ve been going here since 1988.During the right time of the year they fly in stone crabs. You are not a true Arizonian if you haven t been here..... What an institution.... I wish I had this place in Los Angeles.....,0.513976,-0.161250,-0.151098,-0.302935,0.771509
Sometimes you have to explore and try those hole in the places. Sometimes you get junk and other times if your lucky. You get a place like this. I was astounded yes I said it ASTOUNDED. By how authentic and delicious this place is. If you haven t come here you haven t had real Mexican seafood. Trust me you have to come here. Chase out,0.576664,-0.144436,-0.147608,-0.175117,0.770806
I m amazed that this place is in Phoenix. It carries rare and hard to find products from Italy as well as in-house made Italian products. House sausages are insane the deli is truly supherb. Feels like a slice of New York in the desert.,0.584677,0.002777,-0.203439,-0.158738,0.769129


#### perform the same analysis on negative reviews

In [20]:
neg_reviews  = data.ix[data.category == 'low']['Review']
neg_dtm = vectorizer.fit_transform(neg_reviews)
neg_dtm_lsa = lsa.fit_transform(neg_dtm)
neg_dtm_lsa = Normalizer(copy=False).fit_transform(neg_dtm_lsa)
neg_lsa = pd.DataFrame(neg_dtm_lsa, index = neg_reviews, columns = ["component_1","component_2","component_3","component_4","component_5" ])

component 1: taste

In [21]:
neg_lsa.sort_values('component_1', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
Unfortunately I was slightly let down by the yelpers when I went to this resto - food wasn t all that great! For dinner I ordered the grilled burrito and it almost tasted like something I could pull from my freezer and make in the microwave. There wasn t much inside other than beans and cheese - hello what about the guac sour cream and salsas?!?! The slaw was not exciting either. The portions are gigantic but if the foods not very good it s hard to make a dent in your plate. Luckily we ordered a ticoz salad as well and it was hardier and more colorful than the specialty entrees. If it hadn t been for the Passionfruit sangria I would have given this place 2 stars because I was so disappointed. The sangria was perfectly sweet and winey at the same time - filled with fruit and so easy to drink I had to drink 2 cups before I realized I was kind of tipsy. They also offer 3 other sangria flavors but the waiter recommended the passionfruit.In conclusion I would solely recommend this place for drinks and snacks. They have a lounge area on the opposite side of the seated restaurant area that would be perfect for happy hour hang outs with friends.,0.997711,-0.042113,-0.044566,-0.013600,-0.025054
I went to the Phoenix Hard Rock Cafe for lunch on a Friday. It wasn t very busy so I figured I would sit at the bar. The two servers/bartenders were super nice and friendly. So was management that I spoke with so I have nothing negative to say about the service. It was quick and friendly. But the menu! It sucks! It is way overpriced! And nothing I had was good. For $10 I got the worst plate of nachos I have ever had! Sure it was big and plenty to share and even more to throw out but it was awful. Glad I didn t pay another $4 for meat because it was so bad that even meat could not improve it! The beans had no flavor the jalapeno provided the only bit of flavor to the plate. The salsa was pure trash- more live crushed tomatoes and no flavor! You would think in the SW they could at least adapt and go with a better salsa! My other pick was potato skins I love the things. The guy next to me ordered them $9 for a half plate of the tiny-est potatoes I think anyone could skin! I think there were 6 halves but certainly not more than 8 halves all the size of a child s palm! I had a frozen drink called a Pickled Tink $6.75. Awful! Did not come completely mixed and barely pink. Tasted overwhelmingly of the coconut flavor. It was frozen and slushy and weak enough to feed to a kid. If I had one with me I would have let him/her have it seriously. It was my birthday as was maybe 4 other unfortunate souls to be eating there. They gave us each a Sundae with fudge on the whip cream and a small candle. They shouted at the patrons to to shoot Happy Birthday at each of us at separate times and clap. It was pretty weak by the third attempt to get participation. It was nice of my servers to surprise and acknowledge me. Though I was not looking for any handout or commotion. I had a few bites of the sundae and it was even awful! The ice cream was full of ice crystals! So over all I will have to disagree with others and say Applebee s food is much much better and not as expensive! To Applebee s I ll return but to the HRC this is my last visit. Years ago I went to the Phoenix HRC when it was on Camelback if I remember the street correctly. The building & bar was more impressive as was the decor and so was the food. I went on a birthday then too I wonder why they moved to this location? The entry is nothing eye catching or grand as it was then. In fact if the Light Rail driver had not announced it i would not have seen it as I had not seen it on previous trips to and pass the area. Before this stop at the HRC I had been a fan of the place for years. Had once spent many evenings at the Boston HRC when I lived in MA and had fond memories of the old Phoenix location as well. But just as the music we call 

component 2: bad food quality

In [22]:
neg_lsa.sort_values('component_2', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
Americanized Japanese food.,0.388579,0.881848,-0.053888,0.016697,0.261090
Grotesque food vulgar staff blech.,0.412599,0.880058,-0.042579,0.023986,0.229937
Over priced and mediocre food.,0.399957,0.878778,-0.052887,0.015242,0.254469
Excellent food as always,0.396440,0.877795,-0.045229,0.037011,0.262480
Wish the food was hotter.,0.401021,0.877259,-0.045202,0.020116,0.259135
Food is average. The hours are ridiculous! Its always closed????,0.414663,0.876540,-0.034188,0.013540,0.241618
Decent food.,0.411120,0.873419,-0.066183,0.024180,0.251307
Incompetent staff. Medicore food. Not impressed enough to write much other than that you should go elsewhere.,0.434083,0.872555,-0.048565,0.028179,0.216950
Cheap as can be. The coffee is terrible the food is as average as could be.,0.425568,0.871006,-0.052155,0.000139,0.239834


component 3: bad service

In [23]:
neg_lsa.sort_values('component_3', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
It s too confusing to order here.Makes me get a rash.,0.497800,-0.079221,0.818756,0.268656,0.058156
While I was eating a coach roach ran across my table! I immediately stopped eating and threw it in the trash and left disgusted.,0.555847,-0.085816,0.783829,0.260486,-0.037792
Employees gathered in the back chatting so empty no only did they not notice me i had to call their phone on order to get them to come to the front,0.574819,-0.155651,0.778881,0.196273,0.013298
No!I said - NO!!!,0.448808,-0.334794,0.765611,0.294826,0.115773
Waitress took a long time to seat us. Then didn t give us an option on table choice (we didn t want to sit in the doorway) when I asked her to move she told me you can sit over there in such a rude tone. We decided to leave.,0.548806,-0.172134,0.764587,0.283144,-0.066463
Wildly uneven depending on who s in the kitchen and who s at the counter.,0.648154,-0.019735,0.760431,0.023062,-0.026834
disappointing carnitas burrito. no kick in the dish despite being told it had spices. to compound that the seating is ridiculous. There are 2 fox restaurants side by side (blanco and modern burger). I ordered from blanco and assumed I would be allowed to seat in the dining area. As I proceeded to sit this lady comes running and rudely tells me you cannot sit here because you did not order a drink . Mind you I was siting on a crowded high table..... Not at the bar. When I politely old the server that I was waiting for myFood order from blanco she told me my order was to go (though I never asked for that). this despite there being at least 10 empty spots available. plain rude and terrible attitude. not getting my business ever again,0.571540,-0.162217,0.742832,0.304117,0.052348
Went there for breakfast. Only two people I think one was a server. He walked past me twice completely ignoring me. After 15 minutes I finally walked out. Never to return.,0.547842,-0.026946,0.742557,0.323244,-0.208003
OUT OF BUSINESS WHEN I STOPPED BY ON 12/15/10,0.536174,-0.002320,0.741319,0.391244,-0.099427


component 4: many think the food is pricy

In [24]:
neg_lsa.sort_values('component_4', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
Solid. In and out in 45 minutes with a full stomach and not too much damage to the wallet. Nice staff and good service.,0.539655,-0.015979,-0.261369,0.779713,-0.179584
Not very good service. Orders came out wrong. Waited a long time for a glass for my beer. Waited too long for ketchup. Mistakes on check.,0.618219,-0.030360,0.084010,0.751287,-0.213060
Good service bbq was tastee. Overall happy!,0.466080,-0.041541,-0.471130,0.727244,-0.173770
Good sandwiches and quick service.,0.454866,-0.019451,-0.504822,0.705407,-0.200687
While I love Yard House I can t afford to go here when it s not happy hour. They do have good happy hour specials but this location doesn t have a reverse happy hour (after 10pm) on weekends while the one in Scottsdale does. Very deceiving when you expect both to have it! The beer selection is amazing but appetizers are way over-priced unless you are there during happy hour.,0.627803,-0.175758,-0.232820,0.689922,-0.211601
Not bad. Good wait staff. Quick service. She is carrying a gun. I would too if i worked on Van Buren.,0.553420,0.032356,-0.381116,0.689193,-0.269154
So the burgers are good as most people on Yelp have posted but the experience could be better for a number of reasons. We had the $20 dinner special for 2 - where you get 2 burgers 1 order of fries for $20. So when I got the bill I was charged $21.25 and not $20 (this is not including tax) so I was thinking wait the special is advertised for $20 NOT $21.25! The server informed me that I was charged an extra $1.25 because I asked for onions on my burger! WTH!?? Are you serious?? The server never said that I would be charged extra for onions - so be careful when you order here because Zin likes to add deceptive charges - what a Sin Zin! Shysters.I reluctantly give 3 stars because the burger was good and the service was also pretty good. But Zin should include some food on the side with their burgers because when you order a burger you are getting it stand-alone no fries no veggies nutin.If you want to add extra on your burger be careful.,0.702811,-0.097743,-0.116945,0.678603,-0.149415
Appetizer platter was large and tasty.The galaktoboureko dessert was not good.Although the chef is apparently Greek he/she has made lots of compromises. Or did not pay attention to his Yaya.Service was super friendly.,0.621761,-0.118440,-0.366215,0.674603,-0.100906
Here for conference. Convenient location attractive setting. Service was fine (not exceptional). Calamari and Greek fries a virtual sodium fest. Saganaki was good. Three humus platter a disappointment - they only had one flavor left. Over charged us we had to have the bill corrected.,0.557436,-0.130858,-0.448098,0.671700,-0.142016


component 5: chicken

In [25]:
neg_lsa.sort_values('component_5', ascending=False).ix[:20]

,component_1,component_2,component_3,component_4,component_5
Review,,,,,
Chicken fried rice Yum. Friendly employees. Could have been cleaner.,0.304105,-0.196785,-0.252740,-0.398500,0.803814
The yougurt dipping sauce is delish!,0.307831,-0.320947,-0.260124,-0.341788,0.785970
Sigh... the search for anything remotely asian (authentic or fusion) continues. Coming from the Bay Area expectations are high. Yes we adjusted our expectations accordingly for the Phoenix area.The food had a ton of heat but lacked actual flavor.Napa Rolls: napa cabbage wrapped shrimp in a ponzu sauce the dish was cooked well but nothing stoodout eitherGinger Mint Duck Salad: douced with spicey chilli sauce the salad was drowning in a pool of liquid there is no duck taste as at allRed Thai Curry Chicken Wings: plain awful! terrible spicys used and horrible consistency the chicken was marinated in something super salty and then they mixed in a curry sauce that conflicted with the tasteWasabe Tofu: never had tofu with such a heavy batter on it it was not light and delicate beware of the sauce which is 5:1 ratio of wasabi to ponzu you can t even taste the tofu if you use the sauce,0.513052,-0.061188,-0.199045,-0.281162,0.783813
Took a feel bites of my Teriyaki Chicken and had to throw it out. Super sweet. Diabetics...beware!!,0.488693,-0.301444,-0.063701,-0.253418,0.775908
HORRIBLE!!!! I ordered Chicken Fried Rice General Tso s Chicken Orange Beef and egg rolls. Very poor cuts of meat. Terrible Flavor. Cabbage in egg rolls smelled and tasted rotten. I highly recommend going anywhere else.DO NOT EAT THERE!!!,0.447570,-0.292584,-0.116081,-0.334553,0.767252
Avoid the lunch buffet. Excellent Chicken Tikka Masala.,0.422245,-0.192961,-0.268065,-0.366173,0.760614
Ahi salad was poor....meat and dressing had no flavor. But the entrees were excellent - chicken enchiladas and carne adovada - yummy!,0.455434,-0.380065,-0.209904,-0.182207,0.755560
Not a bad option if you are starving at the PHX airport. Ordered the Chicken Lo Mein. The vegetables tasted fresh the chicken was white meat and the service was kind.,0.609307,-0.217425,-0.036625,-0.138234,0.749013
I went with a party of four and was the only one not to order the chicken fried steak - - rather my heart was set on chicken fried chicken. Now I m not a gravy person so I ordered my chicken w/o gravy. This was the most flavorless piece of fried chicken I ve ever eaten. The breading had ZERO seasoning in it and the chicken was not seasoned. While it was moist it required generous amounts of tabasco to make it edible. What a let down. The buttermilk biscuit was good...and that was all I d describe as above average.,0.506404,-0.340906,-0.240537,-0.120772,0.744912
